In [9]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to load and clean CSV data by removing duplicate lines
def load_and_clean_csv(file_path):
    data = pd.read_csv(file_path)
    data = data.drop_duplicates()
    return data

# Load and clean CSV data into a pandas DataFrame
file_path = 'pause_fluxes.csv'
data = load_and_clean_csv(file_path)
data['Cumulative Pause'] = data.groupby('Person')['Pause Before'].cumsum()

# Group data by 'Person'
grouped = data.groupby('Person')

# Define y-axis columns
y_columns = ["Size", "Left Span", "Right Span", "Weight", "RL Ratio", "WS Ratio"]

# Adjust vertical_spacing for better subplot layout
vertical_spacing = 0.01
total_subplots = len(grouped)

fig = make_subplots(rows=total_subplots, cols=1, shared_xaxes=True, vertical_spacing=vertical_spacing)

# Initialize sliders
sliders = [{
    'steps': [],
    'currentvalue': {
        'prefix': 'Measure: '
    },
    'pad': {"t": 50}
}]

# Loop to add traces for each measure and pause points
for measure_index, col in enumerate(y_columns):
    for row_num, (person, group) in enumerate(grouped, start=1):
        # Add pause points for each person, always visible
        if measure_index == 0:  # Add pause points only in the first iteration to avoid duplication
            fig.add_trace(
                go.Scatter(x=group['Cumulative Pause'], y=[0] * len(group), mode='markers',
                           name=f'{person} Pause Before', visible=True, showlegend=(row_num == 1), marker=dict(color="black")),
                row=row_num, col=1
            )

        # Add measure-specific trace, visibility controlled by slider
        visible = (measure_index == 0)  # Only the first measure is visible initially
        fig.add_trace(
            go.Scatter(x=group['Cumulative Pause'], y=group[col], mode='lines+markers',
                       name=f'{person} {col}', visible=visible, showlegend=(row_num == 1)),
            row=row_num, col=1
        )

    # Configure slider steps
    sliders[0]['steps'].append({
        'method': 'update',
        'label': col,
        'args': [{'visible': [True if idx // total_subplots == measure_index or idx % total_subplots == 0 else False for idx in range(len(y_columns) * total_subplots)]},
                 {'title': f'Showing: {col}'}]
    })

# Adjust the height of the figure and update layout with sliders
height_per_subplot = 200
fig.update_layout(height=height_per_subplot * total_subplots, title_text="Subplots for Each Measure and Person",
                  showlegend=True, sliders=sliders)

fig.show()
